# ResNet-18 Implementation

## 1. Setup and Imports
Importing necessary libraries.


In [ ]:
!pip install torchinfo

import torch
import torch.nn as nn
from torchinfo import summary


## 2. ResBlock2D Definition
The ResBlock2D is the fundamental building block of ResNet. It consists of two convolutional layers with a residual connection.


In [ ]:
class ResBlock2D(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResBlock2D, self).__init__()
        # TODO: Define the first convolutional layer with kernel size (3,3) and padding = 1
        self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=stride,padding=1)
        # TODO: Define the first batch normalization layer
        self.bn1 = nn.BatchNorm2d(out_channels)
        # TODO: Define the ReLU activation
        self.relu = nn.ReLU(inplace=True)
        # TODO: Define the second convolutional layer with kernel size (3,3) and padding = 1
        self.conv2 = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=stride,padding=1)
        # TODO: Define the second batch normalization layer
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            # TODO: Define the shortcut layer for dimension matching
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )


    def forward(self, x):
        # TODO: Implement the forward pass
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

## 3. ResNet2D Class Definition
The ResNet2D class defines the overall ResNet-18 architecture. It uses the BasicBlock2D to build the network layers.


In [ ]:
class ResNet2D(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet2D, self).__init__()
        self.in_channels = 3
        # TODO: Define the initial convolutional layer with output 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # TODO: Define the initial batch normalization layer
        self.bn1 = nn.BatchNorm2d(64)
        # TODO: Define the ReLU activation
        self.relu = nn.ReLU(inplace=True)
        # TODO: Create the first ResBLock2D, with in_channels = 64, out_channels = 64
        self.layer1 = self._make_layer(block, 64, layers[0])
        # TODO: Create the second ResBLock2D, with in_channels = 64, out_channels = 128
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        # TODO: Create the third ResBLock2D, with in_channels = 128, out_channels = 256
        self.layer3 = self._make_layer(block, 256, layers[1], stride=2)
        # TODO: Create the fourth ResBLock2D, with in_channels = 256, out_channels = 512
        self.layer4 = self._make_layer(block, 512, layers[1], stride=2)
        # TODO: Define the adaptive average pooling layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # TODO: Define the Flatten Layer
        self.flatten = nn.Flatten()
        # TODO: Define the fully connected layer which outputs the number of classes
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        # TODO: Implement the forward pass
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = self.flatten(out)
        out = self.fc(out)

## 4. ResNet-18 Definition and Testing
Function to create a ResNet-18 model and test it with a random input to ensure it works correctly.


In [ ]:
def ResNet18_2D(num_classes=1000): # ASSUME THE DATASET WILL BE CIFAR-100
    # TODO: Return an instance of ResNet2D with the BasicBlock2D and the appropriate layer configuration for ResNet-18
    return ResNet2D(ResBlock2D, [2, 2, 2, 2], num_classes)

# Example usage:
# TODO: Create an instance of the ResNet-18 model with the specified number of classes
model = ResNet18_2D(num_classes=1000)
print(model)

# TODO: Create a random input tensor with the shape (batch_size, channels, height, width)
input_tensor = torch.randn(1, 3, 224, 224)
# TODO: Pass the input tensor through the model
output_tensor = model(input_tensor)
# TODO: Print the shape of the output tensor to verify it works correctly
print(output_tensor.shape)

ResNet2D(
  (conv1): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (layer1): ResBlock2D(
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential(
      (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): ResBlock2D(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine

In [ ]:
summary(model=model,input_size = (1,3,224,224),)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet2D                                 [1, 1000]                 --
├─Conv2d: 1-1                            [1, 32, 112, 112]         4,704
├─BatchNorm2d: 1-2                       [1, 32, 112, 112]         64
├─ReLU: 1-3                              [1, 32, 112, 112]         --
├─ResBlock2D: 1-4                        [1, 64, 112, 112]         --
│    └─Conv2d: 2-1                       [1, 64, 112, 112]         18,432
│    └─BatchNorm2d: 2-2                  [1, 64, 112, 112]         128
│    └─ReLU: 2-3                         [1, 64, 112, 112]         --
│    └─Conv2d: 2-4                       [1, 64, 112, 112]         36,864
│    └─BatchNorm2d: 2-5                  [1, 64, 112, 112]         128
│    └─Sequential: 2-6                   [1, 64, 112, 112]         --
│    │    └─Conv2d: 3-1                  [1, 64, 112, 112]         2,048
│    │    └─BatchNorm2d: 3-2             [1, 64, 112, 112]         12

In [ ]:
from torchvision.models import resnet18
model = resnet18()
summary(model,input_size = (1,3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│

In [ ]:
from torchvision.models import resnet152
model = resnet152()
summary(model,input_size = (1,3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/5/55/Grace_Hopper.jpg

--2024-06-11 06:12:16--  https://upload.wikimedia.org/wikipedia/commons/5/55/Grace_Hopper.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73746 (72K) [image/jpeg]
Saving to: ‘Grace_Hopper.jpg’

Grace_Hopper.jpg    100%[===================>]  72.02K  --.-KB/s    in 0.03s   

2024-06-11 06:12:16 (2.68 MB/s) - ‘Grace_Hopper.jpg’ saved [73746/73746]



In [ ]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

img = read_image("Grace_Hopper.jpg")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 131MB/s]


military uniform: 10.0%
